<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-17 13:10:00
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: 1.17 L (0.82%)
Current PnL: -31.19 L (-19.5%)
CY Booked + Current PnL: -15.51 L (-9.7%)
-------------------
Total profit:  1.27 L
Total loss:  -32.47 L
-------------------
Total Booked + Current PnL: 11.47 L (8.62%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.95%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.56 L (67.41%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.46,3.60,17.35,21.57,17611.0,3525.0,101505.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.73,48.13,XR,ATH,FINANCE
18,COALINDIA,0.00,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.27,185.0,0.25,1.13,23.94,XY25,ATH,MINING
77,TTKPRESTIG,-0.57,-27.86,38.74,0.09,28166.0,-28072.0,72705.0,770.00,68.84,37.0,M-SC,9.14,250.0,-1.00,0.53,0.06,OX40N,NTT,DURABLES
51,MEDANTA,1.30,-3.14,28.10,24.08,34552.0,-3990.0,122960.0,1486.00,-3.77,56.0,X-SC,11.46,87.0,-0.12,0.89,12.62,XY24,NTT,HEALTHCARE
36,ICICIGI,0.18,2.00,17.00,19.34,34698.0,3994.0,204103.0,2252.93,-14.19,59.0,X-MC,3.03,66.0,0.12,1.48,18.25,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
42,INFY,4.11,-3.22,38.71,34.24,117222.0,-10066.0,302822.0,1972.00,-23.80,32.0,X-LC,4.30,3.0,-0.09,2.19,4.80,X40,NTT,IT
72,TATAELXSI,3.44,-28.13,83.70,32.03,79308.0,-37082.0,94753.0,9161.00,-10.11,40.0,H-SC,6.81,158.0,-0.47,0.69,6.13,OX40N,NTT,IT
83,WHIRLPOOL,3.10,-25.79,145.14,81.92,123648.0,-29606.0,85192.0,2270.00,-56.47,65.0,M-SC,29.65,210.0,-0.24,0.62,19.67,XR,NTT,DURABLES
71,TANLA,2.94,-46.13,307.83,119.69,448985.0,-124906.0,145855.0,1943.92,-45.79,45.0,H-SC,18.96,147.0,-0.28,1.05,15.22,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,-2.03,-29.67,72.41,21.26,56548.0,-32938.0,78094.0,528.00,1835.23,34.0,H-SC,4.21,168.0,-0.58,0.56,7.66,OX40N,NTT,MISC
45,JCHAC,-1.55,-39.46,65.16,-0.01,50418.0,-50430.0,77375.0,2282.00,15726.09,48.0,M-SC,17.14,231.0,-1.00,0.56,7.27,OX40N,BTT,AC
50,MASFIN,-1.46,3.60,17.35,21.57,17611.0,3525.0,101505.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.73,48.13,XR,ATH,FINANCE
55,RAJESHEXPO,-1.01,-67.68,209.57,0.07,92928.0,-92835.0,44342.0,518.00,1525.00,44.0,L-SC,8.97,268.0,-1.00,0.32,10.28,OX40N,NTT,JEWELLERY
59,RELIANCE,-0.90,-8.59,37.07,25.30,68101.0,-17254.0,183709.0,1952.00,-10.80,44.0,X-LC,4.33,21.0,-0.25,1.33,23.06,XY25,BTT,REFINERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.61,3.53,100.83,107.91,154123.0,5210.0,152854.0,1641.55,-6.65,65.0,M-SC,9.47,220.0,0.03,1.11,15.00,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.57,-27.86,38.74,0.09,28166.0,-28072.0,72705.0,770.00,68.84,37.0,M-SC,9.14,250.0,-1.00,0.53,0.06,OX40N,NTT,DURABLES
72,TATAELXSI,3.44,-28.13,83.70,32.03,79308.0,-37082.0,94753.0,9161.00,-10.11,40.0,H-SC,6.81,158.0,-0.47,0.69,6.13,OX40N,NTT,IT
66,SIS,-2.03,-29.67,72.41,21.26,56548.0,-32938.0,78094.0,528.00,1835.23,34.0,H-SC,4.21,168.0,-0.58,0.56,7.66,OX40N,NTT,MISC
40,INDIGOPNTS,-0.45,-30.17,43.23,0.02,52694.0,-52667.0,121892.0,1408.00,85.83,31.0,M-SC,8.33,222.0,-1.00,0.88,6.02,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.00,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.27,185.0,0.25,1.13,23.94,XY25,ATH,MINING
26,FINCABLES,1.61,3.53,100.83,107.91,154123.0,5210.0,152854.0,1641.55,-6.65,65.0,M-SC,9.47,220.0,0.03,1.11,15.00,OX40N,ATH,CABLES
50,MASFIN,-1.46,3.60,17.35,21.57,17611.0,3525.0,101505.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.73,48.13,XR,ATH,FINANCE
1,ABB,-0.60,7.38,35.57,45.57,99923.0,19301.0,280920.0,7934.00,-31.00,69.0,H-MC,3.54,125.0,0.19,2.03,24.75,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.46,3.60,17.35,21.57,17611.0,3525.0,101505.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.73,48.13,XR,ATH,FINANCE
1,ABB,-0.60,7.38,35.57,45.57,99923.0,19301.0,280920.0,7934.00,-31.00,69.0,H-MC,3.54,125.0,0.19,2.03,24.75,AR,NTT,ELECTRICAL
26,FINCABLES,1.61,3.53,100.83,107.91,154123.0,5210.0,152854.0,1641.55,-6.65,65.0,M-SC,9.47,220.0,0.03,1.11,15.00,OX40N,ATH,CABLES
39,INDIAMART,1.13,-4.14,115.66,106.72,136741.0,-5109.0,118227.0,4810.62,-56.84,51.0,H-SC,7.72,140.0,-0.04,0.86,18.27,AR,ATH,MISC
85,ZYDUSLIFE,-0.39,-4.37,41.82,35.63,83818.0,-9149.0,200425.0,1286.17,-16.67,52.0,H-MC,5.60,120.0,-0.11,1.45,11.58,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.05,-17.67,47.14,21.14,88232.0,-40168.0,187171.0,680.00,-19.50,30.0,X-MC,11.94,68.0,-0.46,1.35,0.80,XY24,NTT,PAINTS
8,AWL,0.10,-33.85,137.63,57.19,296915.0,-110393.0,215734.0,485.00,-62.20,31.0,X-SC,17.41,81.0,-0.37,1.56,1.26,XY24,NTT,FMCG
42,INFY,4.11,-3.22,38.71,34.24,117222.0,-10066.0,302822.0,1972.00,-23.80,32.0,X-LC,4.30,3.0,-0.09,2.19,4.80,X40,NTT,IT
73,TCS,1.92,-24.15,56.30,18.55,161516.0,-91353.0,286884.0,4311.59,-31.50,32.0,X-LC,3.61,2.0,-0.57,2.07,2.46,X40,ATH,IT
33,HCLTECH,2.79,-2.49,26.99,23.83,63672.0,-6016.0,235908.0,1908.19,2.56,34.0,X-LC,4.84,8.0,-0.09,1.71,13.18,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.36,0.90,42.41,43.70,106156.0,2240.0,250308.0,735.00,-8.39,54.0,X-MC,1.83,70.0,0.02,1.81,16.09,XY24,BTT,FMCG
65,SIEMENS,-0.78,-14.28,46.42,25.51,74049.0,-26575.0,159520.0,4671.50,33.99,56.0,H-LC,1.93,51.0,-0.36,1.15,17.55,AR,ATH,ELECTRICAL
78,UNITDSPR,1.20,4.15,15.95,20.77,38896.0,9718.0,243862.0,1644.00,-7.70,63.0,X-MC,1.96,64.0,0.25,1.76,11.86,X40N,NTT,BREWERIES
34,HINDUNILVR,-0.39,-10.14,26.49,13.67,61192.0,-26063.0,231000.0,2922.00,-33.31,41.0,X-LC,2.87,9.0,-0.43,1.67,9.05,XY25,NTT,FMCG
36,ICICIGI,0.18,2.00,17.00,19.34,34698.0,3994.0,204103.0,2252.93,-14.19,59.0,X-MC,3.03,66.0,0.12,1.48,18.25,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,0.23,-13.10,51.91,32.01,96320.0,-27968.0,185552.0,17616.87,-41.22,35.0,X-MC,9.32,57.0,-0.29,1.34,0.29,X40,ATH,FMCG
2,ABBOTINDIA,0.68,-11.20,32.96,18.07,52347.0,-20035.0,158820.0,35195.00,-24.45,37.0,X-MC,10.48,58.0,-0.38,1.15,0.68,X40,ATH,PHARMA
3,ACC,0.74,-30.90,137.56,64.15,226176.0,-73531.0,164420.0,3906.00,-47.60,40.0,X-SC,8.36,85.0,-0.33,1.19,0.74,XY24,BTT,CEMENT
13,BERGEPAINT,-0.05,-17.67,47.14,21.14,88232.0,-40168.0,187171.0,680.00,-19.50,30.0,X-MC,11.94,68.0,-0.46,1.35,0.80,XY24,NTT,PAINTS
9,BAJAJHFL,0.02,-23.89,104.28,55.47,193088.0,-58123.0,185163.0,181.50,-34.73,37.0,X-MC,17.39,60.0,-0.30,1.34,0.94,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.02,-30.26,104.82,42.84,47520.0,-19671.0,45335.0,424.00,-54.27,51.0,X-SC,19.45,80.0,-0.41,0.33,9.05,XY24,NTT,MISC
58,RELAXO,1.92,-53.17,225.22,52.31,153107.0,-77179.0,67981.0,1176.00,-50.15,40.0,X-SC,14.19,92.0,-0.50,0.49,1.92,X40N,NTT,FOOTWEAR
12,BATAINDIA,1.09,-45.95,150.15,35.20,104423.0,-59124.0,69546.0,2096.00,-8.88,37.0,X-SC,21.43,91.0,-0.57,0.50,1.09,X40,NTT,FOOTWEAR
51,MEDANTA,1.30,-3.14,28.10,24.08,34552.0,-3990.0,122960.0,1486.00,-3.77,56.0,X-SC,11.46,87.0,-0.12,0.89,12.62,XY24,NTT,HEALTHCARE
35,HONAUT,0.74,-22.82,85.70,43.32,107725.0,-37172.0,125700.0,58357.33,-32.25,39.0,X-SC,10.91,90.0,-0.35,0.91,2.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,2.23,-20.33,39.06,10.78,74782.0,-48864.0,191454.0,452.00,-53.09,49.0,X-LC,18.82,1.0,-0.65,1.38,5.45,X40,NTT,FMCG
73,TCS,1.92,-24.15,56.30,18.55,161516.0,-91353.0,286884.0,4311.59,-31.50,32.0,X-LC,3.61,2.0,-0.57,2.07,2.46,X40,ATH,IT
42,INFY,4.11,-3.22,38.71,34.24,117222.0,-10066.0,302822.0,1972.00,-23.80,32.0,X-LC,4.30,3.0,-0.09,2.19,4.80,X40,NTT,IT
75,TMPV,1.15,-21.79,57.23,22.98,101115.0,-49217.0,176681.0,600.00,-81.44,63.0,X-LC,3.11,4.0,-0.49,1.28,12.95,XY24,NTT,AUTO
33,HCLTECH,2.79,-2.49,26.99,23.83,63672.0,-6016.0,235908.0,1908.19,2.56,34.0,X-LC,4.84,8.0,-0.09,1.71,13.18,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.34,-30.28,90.61,32.89,48774.0,-23378.0,53828.0,1800.00,-353.50,65.0,L-MC,10.90,258.0,-0.48,0.39,48.24,XR,NTT,BANKS
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
67,SONACOMS,1.32,-7.96,51.12,39.09,47597.0,-8050.0,93109.0,804.02,-27.09,64.0,M-MC,6.04,193.0,-0.17,0.67,31.72,AR,ATH,AUTO
50,MASFIN,-1.46,3.60,17.35,21.57,17611.0,3525.0,101505.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.73,48.13,XR,ATH,FINANCE
79,VAIBHAVGBL,2.21,-22.37,107.08,60.77,151943.0,-40878.0,141897.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.64,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
1,ABB,-0.60,7.38,35.57,45.57,99923.0,19301.0,280920.0,7934.00,-31.00,69.0,H-MC,3.54,125.0,0.19,2.03,24.75,AR,NTT,ELECTRICAL
67,SONACOMS,1.32,-7.96,51.12,39.09,47597.0,-8050.0,93109.0,804.02,-27.09,64.0,M-MC,6.04,193.0,-0.17,0.67,31.72,AR,ATH,AUTO
79,VAIBHAVGBL,2.21,-22.37,107.08,60.77,151943.0,-40878.0,141897.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.64,XR,NTT,JEWELLERY
11,BANDHANBNK,0.92,-18.47,136.02,92.42,308906.0,-51456.0,227103.0,400.00,94.68,69.0,H-SC,7.47,174.0,-0.17,1.64,31.25,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.26
1,25,44.66
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.32
MC    30.30
LC    23.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.81
X40      23.28
X40N     15.15
AR        9.04
XR        8.78
XY25      8.75
OX40N     7.30
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.87
H-SC    22.70
X-LC    19.99
X-SC    11.82
M-SC    11.01
H-MC     4.95
M-MC     2.09
H-LC     1.15
L-LC     1.13
M-LC     1.10
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.99,-14.20,52.58
FINANCE,13.33,-13.34,57.94
IT,11.75,-31.71,97.43
MISC,7.08,-32.72,87.01
ELECTRICAL,6.27,-8.11,47.56
PAINTS,5.09,-31.07,50.54
INSURANCE,4.86,-1.02,35.59
PHARMA,4.11,-3.50,36.08
AUTO,3.19,-18.73,59.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3263588.0,21
AR,1321305.0,10
X40,1277926.0,15
XR,1277458.0,12
X40N,1026332.0,10
OX40N,782703.0,10
XY25,409078.0,6
SR,297315.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3652860.0,24
M-SC,1395830.0,14
X-MC,1350726.0,15
X-SC,1170645.0,10
X-LC,1127643.0,12
H-MC,394638.0,3
L-SC,181538.0,2
M-LC,139916.0,1
M-MC,96028.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1244003.0      6
           AR         942286.0      5
           XR         823284.0      7
M-SC       XY24       801521.0      6
X-SC       XY24       605163.0      4
X-LC       X40        564507.0      6
X-MC       X40        539095.0      7
           X40N       408947.0      4
X-SC       X40N       391158.0      4
H-SC       OX40N      345972.0      4
M-SC       OX40N      343803.0      5
H-SC       SR         297315.0      2
X-LC       X40N       226227.0      2
H-MC       XY24       210897.0      1
X-MC       XY25       208296.0      2
X-LC       XY24       207616.0      2
X-MC       XY24       194388.0      2
H-MC       AR         183741.0      2
M-SC       XR         176874.0      2
X-SC       X40        174324.0      2
M-LC       XR         139916.0      1
X-LC       XY25       129293.0      2
L-SC       OX40N       92928.0      1
           XR          88610.0      1
H-LC       AR          74049.0      1
M-SC       AR          73632.0      1
L-MC       XR          48774.0      1
M-MC       XY25        48431.0      1
           AR          47597.0      1
L-LC       XY25        23058.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
